In [1]:
import sys
import os
sys.path = [os.path.abspath(os.path.join(os.curdir,'..'))] + sys.path

In [4]:
from tiledb_xarray import TileDBStore, open_tiledb
import h5py
import xarray as xr
import numpy as np

TB_DS = "../data/local.tiledb"

# store = TileDBStore.open_group(TB_DS)
# print(store)

# vars = store.get_variables()
# print(vars)

t_ds = open_tiledb(TB_DS)
print(t_ds)

print("/\\ tiledb /\\")
print("--------------")
print("\\/ netcdf \\/")
l_ds = xr.open_dataset('../data/local.nc')
print(l_ds)

local_data = l_ds['precipitation_flux'][0:5, 0:10, 10:20].data
tiledb_data = t_ds['precipitation_flux'][0:5, 0:10, 10:20].data

assert np.all(local_data == tiledb_data)
print("*** Yay data match! ***")


<xarray.Dataset>
Dimensions:               (bnds: 2, grid_latitude: 219, grid_longitude: 286, time: 10)
Coordinates:
  * bnds                  (bnds) float32 0.0 0.0
  * grid_latitude         (grid_latitude) float32 22.88 22.66 ... -25.08
  * grid_longitude        (grid_longitude) float32 323.48 323.7 ... 386.18002
  * time                  (time) datetime64[ns] 1854-06-08T12:00:00 ... 1854-06-17T12:00:00
Data variables:
    forecast_period       (time) timedelta64[ns] ...
    forecast_period_bnds  (time, bnds) float64 ...
    precipitation_flux    (time, grid_latitude, grid_longitude) float32 ...
    time_bnds             (time, bnds) datetime64[ns] ...
Attributes:
    _NCProperties:  version=1|netcdflibversion=4.6.1|hdf5libversion=1.10.4
    source:         Data from Met Office Unified Model
    Conventions:    CF-1.5
/\ tiledb /\
--------------
\/ netcdf \/
<xarray.Dataset>
Dimensions:                     (bnds: 2, grid_latitude: 219, grid_longitude: 286, time: 10)
Coordinates:
  * 